In [1]:
import numpy as np
import pandas as pd
from load_and_process import load_online, process_online, load_offline, process_offline, load_CO2, process_CO2
from lmfit import Parameters, report_fit, minimize, fit_report
from model_funcs import sim_single_exp
from parest_funcs import residuals_all_exp
from plotly.subplots import make_subplots
from copy import deepcopy
import plotly.graph_objs as go
import time


In [2]:
#load metadata and create path
path = "C:/Masterarbeit/Messdaten/metadata.xlsx"
exp = "fermentation"
metadata = pd.read_excel(path, index_col = exp)     #read metadata
exp_numbers =[4,5,6,7,8]        # list of experiment numbers used for the parameterestimation

start_dict = {}
for  timestamp, i  in zip(metadata["start"].T, exp_numbers):       # create dictionary with start points out of metadata
    start_dict["ferm{0}".format(i)]= timestamp

end_dict = {}
for  timestamp, i  in zip(metadata["end1"].T, exp_numbers):        # create dictionary with end points from the first day, second day was not considered for the experiment
    end_dict["ferm{0}".format(i)]= timestamp


In [3]:
#load and process online data
#pd.options.mode.chained_assignment = None  # default='warn' for turning annoying panda warning off
path_On = r"C:\Masterarbeit\Messdaten\online"       #path with online data. 
cols = ["PDatTime", "BASET"]        #desired columns to extract
online_raw = load_online(path_On, exp_numbers, cols)
online_dict = process_online(online_raw, start_dict, end_dict)

#load and process offline data
path_Off = r"C:\Masterarbeit\Messdaten\offline"
cols = ["ts","cX","cS","cE","cGly","cP"]        #desired columns to extract
offline_raw = load_offline(path_Off, exp_numbers, cols)
offline_dict = process_offline(offline_raw, start_dict, end_dict)

#load and process CO2 data
path_CO2 = r"C:\Masterarbeit\Messdaten\CO2"
CO2_raw = load_CO2(path_CO2, exp_numbers)
CO2_dict = process_CO2(CO2_raw, start_dict, end_dict)

In [4]:
#individual processing and calculations required for creating parameters later on

#special start because the silicone tube from the base_bottle to the fermenter had a hole, which was recognized too late
special_start_for_online8 = pd.to_datetime("14.12.2020  12:20:16")  
online_dict["ferm8"] = online_dict["ferm8"][(online_dict["ferm8"]["PDatTime"] >= special_start_for_online8 )]  # cut off slice of first time until the hole was fixed


#calculate how much glycerol is produced per ethanol from reliable values
start_etVSgly = {"ferm4" : 5.24, "ferm5" : 4.0, "ferm6" : 4.0, "ferm7": 2.4, "ferm8": 2.5 }     # end and start timepoints for somehow reliable values (high enough and in platau)      
end_etVsgly = {"ferm4" : 10.0, "ferm5" : 10.0, "ferm6" : 6.0, "ferm7" : 5.76, "ferm8" : 6.2 }        

gly_et_list = []  #calculate how much glycerol arise per ethanol,  store it as list entry for every experiment
for ferm, df in offline_dict.items():
    df = df[(df.index >= start_etVSgly[ferm] ) &  (df.index <= end_etVsgly[ferm])]
    gly_et_list.append(np.mean(df["cGly"]/df["cE"]))

# creating individual weighting factors for each experiment with len(dfOff)/len(df2On or dfCO2) as values stored in a dict
len_off_div_on = {ferm : len(offline_dict[ferm])/len(online_dict[ferm]) for ferm in offline_dict.keys()}
len_off_div_CO2 = {ferm : len(offline_dict[ferm])/len(CO2_dict[ferm]) for ferm in offline_dict.keys()}

# get mean pressure values from CO2 measurements and safe them in a dict: CO2_mean_p_dict, In case we want to calculate with p_mean for each experiment later on.
CO2_mean_p_dict = {key : df["p"].mean() for key, df in  CO2_dict.items()}

#drop columns not considered for parameter estimation
[df.drop(["BASET", "PDatTime"], inplace=True, axis=1) for df in online_dict.values()]
[df.drop(["ts","cGly","cP"], inplace=True, axis=1) for df in offline_dict.values()]
[df.drop(["ts","p"], inplace=True, axis=1) for df in CO2_dict.values()]


[None, None, None, None, None]

In [5]:
#creating c_dict containing control variables, y0_dict containing initial values, datasets_dict containing processed data
c_dict = {}
y0_dict = {}
datasets_dict = {}
for ferm in offline_dict.keys():
    x = list(metadata.loc[ferm,["feed_on", "feed_rate","csf", "M_base", "gas_flow", "T"]].values)
    x.append(len_off_div_on[ferm])
    x.append(len_off_div_CO2[ferm])
    x.append(CO2_mean_p_dict[ferm])
    c_dict[ferm] = {"feed_on" : x[0], "feed_rate" : x[1], "csf" : x[2], "M_base" : x[3], "gas_flow" : x[4], "T" : x[5], "wf_on" : x[6], "wf_CO2" : x[7], "pressure" : x[8]}

    y0_dict[ferm] = list(metadata.loc[ferm,["mS0", "mX0", "mE0","V0"]].values)

    datasets_dict[ferm] = [online_dict[ferm], offline_dict[ferm], CO2_dict[ferm]]

#copy datasets_dict only for plotting before dropping dataframes not considered for fitting
datasets_dict_original = deepcopy(datasets_dict)


In [6]:
#drop individual dataframes. In my case is dropped online data of ferm 4,5,6 because i had a bad resolution in base_rate, the correct base recording setting of the sartorius device was carried out from ferm 7.

try:
    if datasets_dict["ferm4"][0].columns in ["base_rate"]:                    # try except for cell rerunability
        datasets_dict["ferm4"].pop(0); datasets_dict["ferm5"].pop(0); datasets_dict["ferm6"].pop(0) # unbrauchbare base values rausschmeißen
except Exception:
    pass


In [7]:
p = Parameters()
p.add("qsmax", value=0.1, min=0.01, max=5.0 , vary = True)     #Maximum glucose uptake rate [g/(g*h)]
#p.add("mumaxE", value=0.17, min=0.12, max=0.19, vary=False)    #Maximum growth rate on ethanol [g/(g*h)]
p.add("qemax", value=0.2361, min=0.15, max=0.35, vary=False)    #Maximum ethanol uptake rate [g/(g*h)]       #value=0.2361, min=0.15, max=0.35, vary=True # value=0.05, min=0.001, max=0.5,  vary=True geht

p.add("base_coef", value= 1, min=0.0001, max =3, vary = True)     #base coefficient [mol/g]      
p.add("qO2max", value= 0.255984, min=0.1, max=0.4, vary = True)    #Maximum oxygen uptake rate [g/(g*h)]  , min=0.1  0.1, 0.4
p.add("qm_max", value=0.01, min=0.0075, max=0.0125, vary = False)  #glucose uptake rate required for maintenance [g/(g*h)]
  
p.add("Ks", value=0.1, min=0.01, max=1,  vary=False)   #Saturation constant, concentration of glucose at µ = 0.5 µmax [g/L]
p.add("Ke", value=0.1, min=0.01, max=1, vary=False)    #Saturation constant, concentration of ethanol at µ = 0.5 µmax [g/L]
p.add("Ki", value=0.1, min=0.01, max=1, vary=False)    #Inhibition constant, glucose inhibits uptake of ethanol [g/L]

p.add("Yxs_ox", value=0.49, min=0.4, max=0.55, vary=False) #Yield biomass per glucose (oxidative growth) [g/g]  
p.add("Yxs_red", value=0.05, min=0.01, max=0.8, vary=False)    #Yield biomass per glucose (reductive growth) [g/g] 
p.add("Yxe_et", value=0.72, min=0.5, max=0.8, vary=False)      #Yield biomass per ethanol [g/g]
p.add("Yxg_glyc", value=0.2, min=0.1, max=0.35, vary=False)    #Yield biomass per glycerol [g/g]              #value=0.2, min=0.1, max=0.35, vary=False

#Biomass composition paramaters, content H,O,N from paper sonnleitner

p.add("HX", value=1.79, min=1.77, max=2.1, vary=False) #Stoichiometric hydrogen content of biomass [mol/mol]
p.add("OX", value=0.57, min=0.54, max=0.63, vary=False) #Stoichiometric oxygen content of biomass [mol/mol]     
p.add("NX", value=0.15, min=0.14, max=0.16, vary=False) #Stoichiometric nitrogen content of biomass [mol/mol]

p.add("g_e", value=np.mean(gly_et_list), min=np.min(gly_et_list), max=np.max(gly_et_list), vary=False) #determined experimentally: formation glycerol per ethanol [g/g]
#hardcode g_e: value: 0.21636282532087284, min = 0.17624196916440354, max = 0.2883951412907231

In [8]:
start = time.time()

result = minimize(residuals_all_exp, p, args=(y0_dict, c_dict, datasets_dict), method='leastsq', nan_policy= "omit")

end = time.time()
print(end - start)

83.57343244552612


In [9]:
report_fit(result)


[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 42
    # data points      = 2597
    # variables        = 3
    chi-square         = 49.9181878
    reduced chi-square = 0.01924371
    Akaike info crit   = -10256.6345
    Bayesian info crit = -10239.0481
[[Variables]]
    qsmax:       1.63854326 +/- 0.00470230 (0.29%) (init = 0.1)
    qemax:       0.2361 (fixed)
    base_coef:   0.00758659 +/- 3.4954e-04 (4.61%) (init = 1)
    qO2max:      0.20956832 +/- 4.7641e-04 (0.23%) (init = 0.255984)
    qm_max:      0.01 (fixed)
    Ks:          0.1 (fixed)
    Ke:          0.1 (fixed)
    Ki:          0.1 (fixed)
    Yxs_ox:      0.49 (fixed)
    Yxs_red:     0.05 (fixed)
    Yxe_et:      0.72 (fixed)
    Yxg_glyc:    0.2 (fixed)
    HX:          1.79 (fixed)
    OX:          0.57 (fixed)
    NX:          0.15 (fixed)
    g_e:         0.2163628 (fixed)
    YCO2x_ox:    1.233382 (fixed)
    YCO2s_ox:    0.604357 (fixed)
    YO2s_ox:     0.4081016 (fixed)
    Yes_red:

In [10]:
#creating data with new model parameters generated from fit
p_new = result.params
fit_dict = {}
t_sim = np.linspace(0,9, 1001) #timeline for simulation

for ferm, y0 in y0_dict.items():
    sim_exp = sim_single_exp(t_sim, y0, p_new, c_dict[ferm])
    fit_dict[ferm] = sim_exp     

In [11]:
#show results for all variables
line_markers = "lines+markers"
line = "lines"
markers = "markers"
dict_longnames = {"ferm4" : "Fermentation 4", "ferm5" : "Fermentation 5", "ferm6" : "Fermentation 6", "ferm7" : "Fermentation 7", "ferm8" : "Fermentation 8"}

color_dict = {"base_rate" : "cyan", "cX" : "red", "cS" : "green", "cE" : "blue", "CO2" : "orange"}


for ferm, df_list in datasets_dict_original.items():
    fig = make_subplots( specs=[[{"secondary_y": True}]])

    for df_dat in df_list:
        
        for column in df_dat:
            secondary_y_flag = column  in ["base_rate", "CO2"]

            fig.add_trace(go.Scatter(x= df_dat.index, y= df_dat[column], name= column, mode = markers, marker = dict(color = color_dict[column], size = 5, symbol = "x")), secondary_y=secondary_y_flag)


    #fit data add
    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["base_rate"], name= "base_rate_fitted", mode = line, marker = dict(color = "cyan")), 
        secondary_y=True,) #"steelblue"
    
    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["cX"], name= "cX_fitted", mode = line, marker = dict(color = "red")), 
        secondary_y=False,) #"crimson"
    
    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["cS"], name= "cS_fitted", mode = line, marker = dict(color = "green")), 
        secondary_y=False,) #"darkgreen"
    

    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["cE"], name= "cE_fitted", mode = line, marker = dict(color = "blue")), 
        secondary_y=False,) #"aquamarine"
    

    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["CO2"], name= "CO2_fitted", mode = line, marker = dict(color = "orange")), 
        secondary_y=True,) #"darkorange"

    fig.update_yaxes(title_text="Biomass(cX) [g/L] , Glucose(cS) [g/L] , Ethanol(cE) [g/L]", secondary_y=False , title_standoff = 20 )
    fig.update_yaxes(title_text="Base consumption rate [ml/h], CO2 [vol. %]", secondary_y=True, title_standoff = 20)
    fig.update_xaxes(title_text= "Time [h]")
    fig.update_layout(title_text = dict_longnames[ferm], title_x=0.5)     
    
    fig.update_layout(legend=dict(x = 1.1))
            
    fig.show()

In [12]:
#Only Offline data  
line_markers = "lines+markers"
line = "lines"
markers = "markers"
dict_longnames = {"ferm4" : "Fermentation 4", "ferm5" : "Fermentation 5", "ferm6" : "Fermentation 6", "ferm7" : "Fermentation 7", "ferm8" : "Fermentation 8"}

color_dict = {"base_rate" : "cyan", "cX" : "red", "cS" : "green", "cE" : "blue", "CO2" : "orange"}
filter_off = ["base_rate", "CO2"]


for ferm, df_list in datasets_dict_original.items():
    fig = make_subplots( specs=[[{"secondary_y": True}]])

    for df_dat in df_list:
        
        for column in df_dat:

            if column not in filter_off:
                secondary_y_flag = column  in ["base_rate", "CO2"]     #filtering only for off-line

                fig.add_trace(go.Scatter(x= df_dat.index, y= df_dat[column], name= column, mode = markers, marker = dict(color = color_dict[column], size = 5, symbol = "x"), line = dict(dash = "dot")), secondary_y=False)

    
    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["cX"], name= "cX_fitted", mode = line, marker = dict(color = "red")), 
        secondary_y=False,) #"crimson"
    
    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["cS"], name= "cS_fitted", mode = line, marker = dict(color = "green")), 
        secondary_y=False,) #"darkgreen"
    

    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["cE"], name= "cE_fitted", mode = line, marker = dict(color = "blue")), 
        secondary_y=False,) #"aquamarine"
    

    fig.update_yaxes(title_text="Biomass(cX) [g/L] , Glucose(cS) [g/L] , Ethanol(cE) [g/L]", secondary_y=False , title_standoff = 20 )
    
    fig.update_xaxes(title_text= "Time [h]")
    fig.update_layout(title_text = dict_longnames[ferm], title_x=0.5)     
    #
    fig.update_layout(legend=dict(x = 1.1))
            
    fig.show() 

In [13]:
#Only Online (base_rate) AND CO2 data  
line_markers = "lines+markers"
line = "lines"
markers = "markers"
dict_longnames = {"ferm4" : "Fermentation 4", "ferm5" : "Fermentation 5", "ferm6" : "Fermentation 6", "ferm7" : "Fermentation 7", "ferm8" : "Fermentation 8"}

color_dict = {"base_rate" : "cyan", "cX" : "red", "cS" : "green", "cE" : "blue", "CO2" : "orange"}
color_dict2 = {"base_rate" : "steelblue", "cX" : "crimson", "cS" : "darkgreen", "cE" : "aquamarine", "CO2" : "darkorange"}

filter_on = ["cX", "cS", "cE"]

for ferm, df_list in datasets_dict_original.items():
    fig = make_subplots( specs=[[{"secondary_y": True}]])

    for df_dat in df_list:
        
        for column in df_dat:

            if column not in filter_on:  #filter 

                fig.add_trace(go.Scatter(x= df_dat.index, y= df_dat[column], name= column, mode = markers, marker = dict(color = color_dict[column], size = 5, symbol = "x")), secondary_y= False)


    #fit data add
    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["base_rate"], name= "base_rate_fitted", mode = line, marker = dict(color = "cyan")), 
        secondary_y=False,) #"steelblue"
    
    fig.add_trace( 
        go.Scatter(x= t_sim, y= fit_dict[ferm]["CO2"], name= "CO2_fitted", mode = line, marker = dict(color = "orange")), 
        secondary_y=False,) #"darkorange"


    
    fig.update_yaxes(title_text="CO2 [vol. %], Base consumption rate [mL/h]", secondary_y=False, title_standoff = 20)
    fig.update_xaxes(title_text= "Time [h]")
    fig.update_layout(title_text = dict_longnames[ferm], title_x=0.5)     
    
    fig.update_layout(legend=dict(x = 1.1))
            
    fig.show()

In [14]:
#Individual plotting for my master thesis
#for show PID imporvement
from load_and_process import process_online_optional_filter

exp_nums = [2,4,5,6]
cols2 = ["PDatTime","STIRR", "pO2"]
path_on2 = r"C:\Masterarbeit\Messdaten\online"
online_raw2 = load_online(path_on2, exp_nums, cols2)
online_dict2 = process_online_optional_filter(online_raw2)

In [15]:
from load_and_process import process_online_optional_filter, process_offline_optional_filter ,process_CO2_optional_filter

online_new = process_online_optional_filter(online_raw, start_dict = start_dict)    #end_dict = end_dict
off_new = process_offline_optional_filter(offline_raw, start_dict = start_dict)     #end_dict = end_dict
CO2_new = process_CO2_optional_filter(CO2_raw, start_dict = start_dict)             #end_dict = end_dict

[df.drop(["BASET", "PDatTime"], inplace=True, axis=1) for df in online_new.values()]
[df.drop(["ts"], inplace=True, axis=1) for df in off_new.values()]
[df.drop(["ts"], inplace=True, axis=1) for df in CO2_new.values()]

[None, None, None, None, None]

In [16]:
#Just to show some measurement data without fitting
color_dict = {"base_rate" : "cyan", "cX" : "red", "cS" : "green", "cE" : "blue", "CO2" : "orange", "cP" : "grey", "cGly" : "brown"}
line_markers = "lines+markers"
markers = "markers"
X = "x"
line = "lines"
dict_longnames = {"ferm4" : "Fermentation 4", "ferm5" : "Fermentation 5", "ferm6" : "Fermentation 6", "ferm7" : "Fermentation 7", "ferm8" : "Fermentation 8"}

#swap for columns of desire
#filter_cols = ["cP"] #nur ohne phosphate
#filter_cols = ["cP", "cGly"] #nur stannis offline vom schätzer
#filter_cols = ["cX", "cS", "cE", "cGly"] #alles außer phosphor
filter_cols = [ "cE", "cGly"] 

for ferm, df in off_new.items():
    fig = make_subplots( specs=[[{"secondary_y": True}]])

    for column in df :
        if column not in filter_cols:
            secondary_y_flag = column  in ["cGly", "cP", "cE", "cS"]
            if ferm == "ferm7" and column == "cX":
                fig.add_trace( go.Scatter(x= df.index.values[:-2], y= df[column].values[:-2], mode = markers, name = column, marker = dict(color = color_dict[column], size = 5, symbol="x"), line = dict(dash = "solid")), secondary_y= secondary_y_flag)  #letzte 2 cx raus

            else:
                fig.add_trace( go.Scatter(x= df.index, y= df[column], mode = markers, name = column, marker = dict(color = color_dict[column], size = 5, symbol="x"), line = dict(dash = "solid")), secondary_y= secondary_y_flag)
            
    fig.add_trace( go.Scatter(x= CO2_new[ferm].index, y= CO2_new[ferm]["CO2"], mode = markers, name = column, marker = dict(color = "orange", size = 5, symbol = "x")), secondary_y= True)
    

    
    fig.update_yaxes(title_text="Biomass(cX) [g/L] , Glucose(cS) [g/L] ", secondary_y=False , title_standoff = 20 )
    fig.update_yaxes(title_text="Glycerol(cGly) [g/L], Ethanol(cE) [g/L], Phosphate(cP) [g/L]", secondary_y=True, title_standoff = 20) #, tickangle = 45
    fig.update_xaxes(title_text= "Time [h]")
    fig.update_layout(title_text = dict_longnames[ferm], title_x=0.5)     
    
    fig.update_layout(legend=dict(x = 1.1))    
    
    fig.show()

In [17]:
#calculate residuals
from get_residuals import get_residuals
res_and_data = get_residuals(p_new, y0_dict, c_dict, datasets_dict)

In [18]:
#Remse single
REMSE_all_dict = {}
for ferm, lis in res_and_data.items():
    REMSE_dict = {}
    for df in lis:
   
        if df.columns.values[0] in ["cX"]:
            REMSE_dict["cX_RMSE"] = np.sqrt(np.mean(df["cX_residuals"]**2))
            REMSE_dict["cS_RMSE"] = np.sqrt(np.mean(df["cS_residuals"]**2))
            REMSE_dict["cE_RMSE"] = np.sqrt(np.mean(df["cE_residuals"]**2))
        if df.columns.values[0] in ["base_rate"]:
            REMSE_dict["base_rate_RMSE"] = np.sqrt(np.mean(df["base_rate_residuals"]**2))
        if df.columns.values[0] in ["CO2"]:
            REMSE_dict["CO2_RMSE"] = np.sqrt(np.mean(df["CO2_residuals"]**2))
    
    REMSE_all_dict[ferm] = REMSE_dict
REMSE_all_dict

{'ferm4': {'cX_RMSE': 0.5543312585979294,
  'cS_RMSE': 0.6035074515501244,
  'cE_RMSE': 0.4051189862264529,
  'CO2_RMSE': 0.8367862793192191},
 'ferm5': {'cX_RMSE': 0.30496939488712616,
  'cS_RMSE': 0.3010666098052961,
  'cE_RMSE': 0.5496581638936563,
  'CO2_RMSE': 0.6263418106625761},
 'ferm6': {'cX_RMSE': 0.3295168578909516,
  'cS_RMSE': 0.28278654274526643,
  'cE_RMSE': 0.3337252098321189,
  'CO2_RMSE': 0.6144749800074365},
 'ferm7': {'base_rate_RMSE': 0.28511735617266604,
  'cX_RMSE': 0.573594838387193,
  'cS_RMSE': 0.1806884803804214,
  'cE_RMSE': 0.28948183599131516,
  'CO2_RMSE': 0.7158733902202585},
 'ferm8': {'base_rate_RMSE': 0.1932296219647481,
  'cX_RMSE': 0.4535260965779782,
  'cS_RMSE': 0.497992044019653,
  'cE_RMSE': 0.3931893237475208,
  'CO2_RMSE': 0.7719059864506608}}

In [19]:
#create residuals frame for all experiments
pd.set_option('display.max_rows', 1000)
online_res = pd.concat([res_and_data["ferm7"][0], res_and_data["ferm8"][0]])
offline_res = pd.concat([res_and_data["ferm4"][0], res_and_data["ferm5"][0], res_and_data["ferm6"][0], res_and_data["ferm7"][1], res_and_data["ferm8"][1]])
CO2_res = pd.concat([res_and_data["ferm4"][1], res_and_data["ferm5"][1], res_and_data["ferm6"][1], res_and_data["ferm7"][2], res_and_data["ferm8"][2]])

In [20]:
#Remse all 
online_remse_all = np.sqrt(np.mean(online_res["base_rate_residuals"]**2))
cX_remse_all = np.sqrt(np.mean(offline_res["cX_residuals"]**2))
cS_remse_all = np.sqrt(np.mean(offline_res["cS_residuals"]**2))
cE_remse_all = np.sqrt(np.mean(offline_res["cE_residuals"]**2))
CO2_remse_all = np.sqrt(np.mean(CO2_res["CO2_residuals"]**2))

print(online_remse_all, cX_remse_all, cS_remse_all, cE_remse_all, CO2_remse_all)

0.2501316624737784 0.4612315164173869 0.39860390820218566 0.4011874336953883 0.7179720217827729


In [21]:
#bias single
bias_all_dict = {}
for ferm, lis in res_and_data.items():
    bias_dict = {}
    for df in lis:
   
        if df.columns.values[0] in ["cX"]:
            bias_dict["cX_bias"] = np.mean(df["cX_residuals"])
            bias_dict["cS_bias"] = np.mean(df["cS_residuals"])
            bias_dict["cE_bias"] = np.mean(df["cE_residuals"])
        if df.columns.values[0] in ["base_rate"]:
            bias_dict["base_rate_bias"] = np.mean(df["base_rate_residuals"])
        if df.columns.values[0] in ["CO2"]:
            bias_dict["CO2_bias"] = np.mean(df["CO2_residuals"])
    
    bias_all_dict[ferm] = bias_dict
bias_all_dict

{'ferm4': {'cX_bias': -0.4873380940259963,
  'cS_bias': -0.05831519616057048,
  'cE_bias': -0.13841464766621125,
  'CO2_bias': 0.7544619496148713},
 'ferm5': {'cX_bias': -0.15734194230696943,
  'cS_bias': 0.14257480350798463,
  'cE_bias': 0.04029156958833052,
  'CO2_bias': 0.6144963323625615},
 'ferm6': {'cX_bias': -0.10612522807684532,
  'cS_bias': -0.13117436285735307,
  'cE_bias': -0.051079969301809705,
  'CO2_bias': 0.6057480035863119},
 'ferm7': {'base_rate_bias': 0.15429055007987513,
  'cX_bias': -0.09232164310992504,
  'cS_bias': -0.033288500555791314,
  'cE_bias': 0.12729312269366347,
  'CO2_bias': 0.6852486603364821},
 'ferm8': {'base_rate_bias': -0.15269510896677765,
  'cX_bias': 0.3133535895608991,
  'cS_bias': -0.2794654649819778,
  'cE_bias': -0.02108536217274236,
  'CO2_bias': 0.7417171219008329}}

In [22]:
#bias all 
online_bias_all = np.mean(online_res["base_rate_residuals"])
cX_bias_all = np.mean(offline_res["cX_residuals"])
cS_bias_all = np.mean(offline_res["cS_residuals"])
cE_bias_all = np.mean(offline_res["cE_residuals"])
CO2_bias_all = np.mean(CO2_res["CO2_residuals"])

print(online_bias_all, cX_bias_all, cS_bias_all, cE_bias_all, CO2_bias_all)

0.023503760426863234 -0.10078379488841689 -0.07056804912169339 -0.0020762213071587334 0.6821956286970411


In [23]:
#STDDEV all
online_STDDEV_all = np.sqrt(np.mean((online_res["base_rate_residuals"]-online_bias_all)**2))
cX_STDDEV_all = np.sqrt(np.mean((offline_res["cX_residuals"]-cX_bias_all)**2))
cS_STDDEV_all = np.sqrt(np.mean((offline_res["cS_residuals"]-cS_bias_all)**2))
cE_STDDEV_all = np.sqrt(np.mean((offline_res["cE_residuals"]-cE_bias_all)**2))
CO2_STDDEV_all = np.sqrt(np.mean((CO2_res["CO2_residuals"]-CO2_bias_all)**2))

print(online_STDDEV_all, cX_STDDEV_all, cS_STDDEV_all, cE_STDDEV_all, CO2_STDDEV_all)

0.24902494215980217 0.45008570119986213 0.3923075656640014 0.4011820612392524 0.22381453985273167


In [24]:
#STDDEV all n-1
online_STDDEV_all = np.std(online_res["base_rate_residuals"], ddof = 1)
cX_STDDEV_all = np.std(offline_res["cX_residuals"], ddof = 1)         
cS_STDDEV_all = np.std(offline_res["cS_residuals"], ddof = 1) 
cE_STDDEV_all = np.std(offline_res["cE_residuals"], ddof = 1) 
CO2_STDDEV_all = np.std(CO2_res["CO2_residuals"], ddof = 1) 

print(online_STDDEV_all, cX_STDDEV_all, cS_STDDEV_all, cE_STDDEV_all, CO2_STDDEV_all)

0.2497649881959287 0.4526357822369045 0.39450538434897126 0.4035077738312737 0.22386631881001268


In [25]:
#STDDEV single n-1
STDDEV_res_all_dict_emp = {}
for ferm, lis in res_and_data.items():
    STDDEV_res_dict = {}
    for df in lis:
   
        if df.columns.values[0] in ["cX"]:
            STDDEV_res_dict["cX_STDDEV"] = np.std(df["cX_residuals"], ddof= 1)
            STDDEV_res_dict["cS_STDDEV"] = np.std(df["cS_residuals"], ddof= 1)
            STDDEV_res_dict["cE_STDDEV"] = np.std(df["cE_residuals"], ddof= 1)
        if df.columns.values[0] in ["base_rate"]:
            STDDEV_res_dict["base_rate_STDDEV"] = np.std(df["base_rate_residuals"], ddof= 1)
        if df.columns.values[0] in ["CO2"]:
            STDDEV_res_dict["CO2_STDDEV"] = np.std(df["CO2_residuals"], ddof= 1)
    
    STDDEV_res_all_dict_emp[ferm] = STDDEV_res_dict
STDDEV_res_all_dict_emp

{'ferm4': {'cX_STDDEV': 0.2722981303057197,
  'cS_STDDEV': 0.6191703064154562,
  'cE_STDDEV': 0.39322635152469976,
  'CO2_STDDEV': 0.3624642927262922},
 'ferm5': {'cX_STDDEV': 0.26928694792348523,
  'cS_STDDEV': 0.27285457460378926,
  'cE_STDDEV': 0.5650504220551872,
  'CO2_STDDEV': 0.12137590593439795},
 'ferm6': {'cX_STDDEV': 0.32156060132572334,
  'cS_STDDEV': 0.2582326663925618,
  'cE_STDDEV': 0.3406086354961157,
  'CO2_STDDEV': 0.10333652483630051},
 'ferm7': {'base_rate_STDDEV': 0.241008607554041,
  'cX_STDDEV': 0.5808231480434187,
  'cS_STDDEV': 0.18220925700943266,
  'cE_STDDEV': 0.26674686262407665,
  'CO2_STDDEV': 0.20735023801814045},
 'ferm8': {'base_rate_STDDEV': 0.11924504986702039,
  'cX_STDDEV': 0.337370285953861,
  'cS_STDDEV': 0.4241331877528027,
  'cE_STDDEV': 0.40400629964507373,
  'CO2_STDDEV': 0.21397030238868148}}